## Parsing weather files 

The end result of parsing should be to get 3 csv files: 1) the historical daily highs/lows (alminac), 2) the historical monthly average precipitation (precip_normal), 3) the recent daily highs/lows and precipitation (weather), 4) metadata file to tie the files together: filenames, unique place identifier, units for each file, etc.

In [1]:
import pandas as pd

#### Final Format 

In [2]:
almanac_columns = ['month', 'day', 'T_high_avg', 'T_low_avg', 'T_max', 
                   'T_min', 'T_max_yr', 'T_min_yr', 'period_T_high', 'period_T_low']
df_alminac = pd.DataFrame(columns=almanac_columns, index=range(366))

In [3]:
precip_normal_columns = ['month', 'h_all_precip', 'h_rain', 'h_snow']
df_precip_normal = pd.DataFrame(columns=precip_normal_columns, index=range(12))

In [4]:
weather_columns = ['date', 'T_high', 'T_low', 'h_all_precip']
df_weather = pd.DataFrame(columns=weather_columns, index=range(365))

#### Parsing the files with daily data 

In [5]:
def parse_daily_file(old_filepath, new_filepath, col_dict):
    f = open(old_filepath, 'r')
    s = f.readlines()
    f.close()
    obj = {}
    n = 0
    
    while s[n] != '\n':
        prop = s[n].split(',')[0][1:-1]
        val = s[n].split(',')[1][1:-2]
        obj[prop] = val
        n+=1
    
    # skip the data legend
    n+=1
    while s[n] != '\n': n+=1
    
    df = pd.DataFrame.from_csv(old_filepath, header=n+1, index_col=None)

    # If this doesn't work I could use the column position
    for c in df.columns: 
        if c in col_dict:
            # put the snow height in mm
            if '(cm)' in c: df[c] = 10.*df[c]
            df[col_dict[c]] = df[c]
        del df[c]

    df.to_csv(new_filepath, index=False)
#     obj['fname'] = new_filepath
#     return obj

In [8]:
almanac_col_dict = {'Month': 'month', 'Day': 'day', 
                    'Average Max. Temp. (°C)': 'T_high_avg', 
                    'Average Min. Temp. (°C)': 'T_low_avg', 
                    'Highest Temp. (°C)': 'T_max', 
                    'Lowest Temp. (°C)': 'T_min', 
                    'Highest Temp.Year': 'T_max_year', 
                    'Lowest Temp. Year': 'T_min_year',
                    'Highest Temp. Period': 'period_T_high', 
                    'Lowest Temp. Period': 'period_T_low'}
old_filepath = "../data/almanac/to.csv"
new_filepath = "../data/almanac/to_clean.csv"
parse_daily_file(old_filepath, new_filepath, almanac_col_dict)    

#### Parsing the precipitation files 

In [9]:
def parse_precipitation(old_filepath, new_filepath):
    f = open(old_filepath, 'r')
    s = f.readlines()
    f.close()

    key = s[3].split(',')
    val = s[4].split(',')

    metadata = {key[m].replace('"', ''): val[m].replace('"', '') for m, _ in enumerate(key)}

    n = 0
    while s[n] != '"Precipitation"\n': n+=1

    r_1 = s[n+1].split(',')
    r_2 = s[n+2].split(',')
    r_3 = s[n+3].split(',')

    if not (r_1[0] == '"Rainfall (mm)"' and
            r_2[0] == '"Snowfall (cm)"' and 
            r_3[0] == '"Precipitation (mm)"'):
        print 'error'

    cols = ['month', 'h_all_precip', 'h_rain', 'h_snow']
    df = pd.DataFrame(columns=cols, index=range(12))

    df['month'] = range(1, 13)
    df['h_all_precip'] = [float(x.replace('"', '')) for x in r_3[1:13]]
    df['h_rain'] = [float(x.replace('"', '')) for x in r_1[1:13]]
    df['h_snow'] = [10.*float(x.replace('"', '')) for x in r_2[1:13]]

    df.to_csv(new_filepath, index=False)
    return metadata

In [10]:
old_filepath = "../data/precipitation/to.csv"
new_filepath = "../data/precipitation/to_clean.csv"
obj = parse_precipitation(old_filepath, new_filepath)

In [11]:
obj

{'CLIMATE_ID': '6158350',
 'ELEVATION': '112.5 m',
 'LATITUDE': " 43\xb040'00.000 N",
 'LONGITUDE': " 79\xb024'00.000 W",
 'PROVINCE': 'ON',
 'STATION_NAME': 'TORONTO',
 'TC_ID\n': '\n',
 'WMO_ID': '71266'}

#### Parse Weather File 

In [12]:
weather_columns = ['date', 'T_high', 'T_low', 'h_all_precip']
df_weather = pd.DataFrame(columns=weather_columns, index=range(365))

In [16]:
old_filepath = '../data/daily_weather/to_intl_a.csv'
new_filepath = '../data/daily_weather/to_intl_a_clean.csv'
weather_col_dict = {'Date/Time': 'date', 'Max Temp (°C)': 'T_high', 
                    'Min Temp (°C)': 'T_low', 'Total Rain (mm)': 'h_rain',
                    'Total Snow (cm)': 'h_snow', 'Total Precip (mm)': 'h_all_precip'}

In [17]:
parse_daily_file(old_filepath, new_filepath, weather_col_dict)

#### Save a lookup table for the files 

In [ ]:
file_lookup = [{'city':'Toronto', 'prov': 'ON', 'daily_weather': '/daily_weather/to_intl_a_clean.csv',
                'almanac': '/almanac/to_clean.csv', 'precipitation': '/precipitation/to_clean.csv'},
               {'city': 'Yellowknife', 'prov': 'NWT', 'daily_weather': '/daily_weather/yk_clean.csv',
                'almanac': '/almanac/yk_clean.csv', 'precipitation': '/precipitation/yk_clean.csv'}]